# Feast Prompt Templates

This example shows how to bind a LangChain prompt template to an existing Feast feature store
and have the prompt formatted with information from the (online) store.

The prompt template used here, specialized to retrieval from Feast, is another application of the "convertor prompt template" covered and illustrated in another tutorial. Please check that out for more background.

_Note_: to run this code, you need to go first complete the **"Feast/Cassandra setup"** instructions given earlier.

## Initialization of the feature store

First get your feature store ready and queryable:

In [1]:
from feast import FeatureStore

feast_repo_path = "feast_store/user_features/feature_repo/"
store = FeatureStore(repo_path=feast_repo_path)

## Creation of the prompt template

You need the `FeastReaderPromptTemplate` to seamlessly bind the feature store to a prompt:

In [2]:
from langchain.prompts.database import FeastReaderPromptTemplate

... and of course a prompt string:

In [3]:
ecommercePrompt = """You are a helpful assistant for
customers of an e-commerce website.
You will give them a suggestion on what to do,
based on their profile and pattern of usage so far.
The customer information is summarized by the following values:

> Customer age:               {age}
> Number of purchases so far: {purchases}
> Site visits (per month):    {visit_frequency}
> Active cart now? (1 = yes): {active_cart}

USER QUESTION: {user_question}

YOUR SUGGESTION: """

Create the prompt template. Note that you have to specify several required settings:
- the feature store;
- the template string to format with the variables;
- the names of the "regular" input variables (i.e. those _not_ coming through a Feast lookup)
- a dictionary specifying, for each variable appearing in the prompt string, the feature view and the feature name for the Feast lookup.

In [4]:
ecommerceTemplate = FeastReaderPromptTemplate(
    feature_store=store,
    template=ecommercePrompt,
    input_variables=["user_question"],
    field_mapper={
        'age': ('user_data', 'age'),
        'purchases': ('user_data', 'purchases'),
        'visit_frequency': ('user_data', 'visit_frequency'),
        'active_cart': ('active_cart', 'active_cart'),
    },
)

## Formatting the template

### Full formatting

To format the template, the "join keys" for the Feast lookup must be supplied together with the other regular variables that appear in the template string directly.

The Feast prompt template will do the magic of using the join keys, turning them into the requested features, and placing them in the template string:

In [5]:
print(ecommerceTemplate.format(
    user_name='marilyn', user_question='How do I remove an item from the cart?'
))

You are a helpful assistant for
customers of an e-commerce website.
You will give them a suggestion on what to do,
based on their profile and pattern of usage so far.
The customer information is summarized by the following values:

> Customer age:               55
> Number of purchases so far: 12
> Site visits (per month):    0.17000000178813934
> Active cart now? (1 = yes): 0

USER QUESTION: How do I remove an item from the cart?

YOUR SUGGESTION: 


### Null values

By default, features that are not found will result in a literal `None` value:

In [6]:
print(ecommerceTemplate.format(
    user_name='rachel', user_question='How do I remove an item from the cart?'
))

You are a helpful assistant for
customers of an e-commerce website.
You will give them a suggestion on what to do,
based on their profile and pattern of usage so far.
The customer information is summarized by the following values:

> Customer age:               None
> Number of purchases so far: None
> Site visits (per month):    None
> Active cart now? (1 = yes): None

USER QUESTION: How do I remove an item from the cart?

YOUR SUGGESTION: 


You can choose to raise an error if a null feature is encountered with `admit_nulls=False`:

In [7]:
fussyTemplate = FeastReaderPromptTemplate(
    feature_store=store,
    template="Question: {user_question}, purchases: {purchases}",
    input_variables=["user_question"],
    field_mapper={
        'purchases': ('user_data', 'purchases'),
    },
    admit_nulls=False,
)

try:
    # Note: we pick a user_name that is not found on the feature store
    fussyTemplate.format(user_name="rachel", user_question="Do I exist?")
except Exception as e:
    print(f"Exception => {str(e)}")

Exception => Null feature value found for "user_data:purchases"


Alternatively, you can specify _per-variable_ whether nulls are admitted, and even provide a default value for that case:

In [8]:
fussyTemplate = FeastReaderPromptTemplate(
    feature_store=store,
    template="Question: {user_question}, purchases: {purchases}. visits: {visit_frequency}.",
    input_variables=["user_question"],
    field_mapper={
        'purchases': ('user_data', 'purchases', True),
        'visit_frequency': ('user_data', 'visit_frequency', True, "(no visits)"),
    },
    admit_nulls=False,  # this will be overridden per-variable if provided above
)

fussyTemplate.format(user_name="rachel", user_question="Do I exist?")

'Question: Do I exist?, purchases: None. visits: (no visits).'

### Partialing the template

In some cases you want to "pin" some of the input variables for a prompt template, getting a partially-ready object that you will later `format` by passing the remaining input parameters. Feast prompt templates let you do that (regardless whether by partialing on the Feast-bound or the regular inputs):

In [9]:
eCommPartial = ecommerceTemplate.partial(user_name='rodolfo')
print(eCommPartial.format(user_question='When is the next special offer?'))

You are a helpful assistant for
customers of an e-commerce website.
You will give them a suggestion on what to do,
based on their profile and pattern of usage so far.
The customer information is summarized by the following values:

> Customer age:               21
> Number of purchases so far: 323
> Site visits (per month):    8.59000015258789
> Active cart now? (1 = yes): 1

USER QUESTION: When is the next special offer?

YOUR SUGGESTION: 


In [10]:
eCommPartial2 = ecommerceTemplate.partial(user_question='When is the next special offer?')
print(eCommPartial2.format(user_name='rodolfo'))

You are a helpful assistant for
customers of an e-commerce website.
You will give them a suggestion on what to do,
based on their profile and pattern of usage so far.
The customer information is summarized by the following values:

> Customer age:               21
> Number of purchases so far: 323
> Site visits (per month):    8.59000015258789
> Active cart now? (1 = yes): 1

USER QUESTION: When is the next special offer?

YOUR SUGGESTION: 
